In [1]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import glob
from pymatreader import read_mat

import torch
from torch.utils.data import Dataset, ConcatDataset
from torchsummary import summary
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

import warnings
warnings.filterwarnings('ignore')
%matplotlib widget

In [2]:
# dataset class

class SeqDataset(Dataset):
    
    def __init__(self, root, seq_length, is_train, transform=None):
        self.transform = transform
        self.seqs = []
        self.seq_labels = []
        self.class_names = os.listdir(root)
        self.class_names.sort()
        self.numof_classes = len(self.class_names)
        self.seq_length = seq_length
        self.is_train = is_train

        for (i,x) in enumerate(self.class_names):
            temp = glob.glob(os.path.join(root, x, '*'))
            temp.sort()
            self.seq_labels.extend([i]*len(temp))
            for t in temp:
                df = pd.read_csv(t, header=None)
                tensor = preprocess(df)
                self.seqs.append(tensor)

    def __getitem__(self, index):
        seq = self.seqs[index]
        if self.transform is not None:
            seq = self.transform(seq, is_train=self.is_train, seq_length=self.seq_length)
        return {'seq':seq, 'label':self.seq_labels[index]}

    def __len__(self):
        return len(self.seqs)

def preprocess(df: pd.DataFrame)->np.ndarray:
    mat = df.T.values
    mat = standardization(mat, axis=1)

    return mat

def standardization(a, axis=None, ddof=0):
    a_mean = a.mean(axis=axis, keepdims=True)
    a_std = a.std(axis=axis, keepdims=True, ddof=ddof)
    a_std[np.where(a_std==0)] = 1

    return (a - a_mean) / a_std

def add_noise(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    data_noisy = data + noise

    return data_noisy.astype(np.float32)

def time_shift(data, shift):
    data_shifted = np.roll(data, shift)

    return data_shifted

def transform(array, is_train, seq_length):
    if is_train:
        _, n = array.shape
        s = random.randint(0, n-seq_length)
        ts = array[:,s:s+seq_length]
        ts = add_noise(ts).astype(np.float32)
        if random.randint(0,1):
            ts_r = ts[:,::-1].copy()
            return ts_r
        return ts
    else:
        ts = array[:,:seq_length].astype(np.float32)
        return ts

In [3]:
# train dataset

train_datasets = []
train_ids = ['subject0', 'subject1', 'subject2', 'subject3']

for train_id in train_ids:
    train_dir = os.path.join('Motion Decoding Using Biosignals', 'dataset', 'train', train_id)
    dataset = SeqDataset(root=train_dir, seq_length=250, is_train=True, transform=transform)
    train_datasets.append(dataset)

train_dataset = ConcatDataset(train_datasets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)

In [4]:
# test dataset

batch_size = 32

test_dir = os.path.join('Motion Decoding Using Biosignals', 'dataset', 'val', 'subject4')

test_dataset = SeqDataset(root=test_dir, seq_length=250, is_train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [5]:
# modeling conv1d

class conv1d(torch.nn.Module):
    def __init__(self, num_channels, num_classes):
        super(conv1d, self).__init__()
        self.conv1 = torch.nn.Conv1d(num_channels, 32, kernel_size=7, stride=1, padding=3)
        self.conv2 = torch.nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.conv3 = torch.nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1) 
        
        self.bn1 = torch.nn.BatchNorm1d(32)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.bn3 = torch.nn.BatchNorm1d(128)
        
        self.maxpool = torch.nn.MaxPool1d(kernel_size=3, stride=2)
        
        self.dropout = torch.nn.Dropout(p=0.5)
        
        self.gap = torch.nn.AdaptiveAvgPool1d(1)
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.maxpool(x)

        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.gap(x)
        x = x.squeeze(2)

        x = self.dropout(x)

        x = self.fc(x)

        return x
        
num_channels = 72 
num_classes = 3  

model = conv1d(num_channels, num_classes)

input_data = torch.randn(32, num_channels, 300)
output_data = model(input_data)

In [7]:
# model training

def train(log_interval, model, device, train_loader, optimizer, epoch, iteration):

    model.train()
    criterion = torch.nn.CrossEntropyLoss()

    for sample_batched in train_loader:
        data, target = sample_batched['seq'].to(device), sample_batched['label'].to(device)

        optimizer.zero_grad()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        iteration += 1

    #    if iteration % log_interval == 0:
    #        print('Train Accracy: {3:5.2f}%  train_loss: {2:.6f} \n'.format(epoch, iteration, loss.item(), 100.*correct/float(len(sample_batched['label']))))
            
    return iteration

def val(model, device, test_loader):

    model.eval()
    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for sample_batched in test_loader:
            data, target = sample_batched['seq'].to(device), sample_batched['label'].to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= float(len(test_loader.dataset))
    correct /= float(len(test_loader.dataset))
    
    print('Validation Accuracy: {0:.2f}%  Test Loss: {1:.6f} \n'.format(100. * correct, test_loss))

    return test_loss, 100. * correct

def evaluate(model, device, test_loader):

    preds = []
    trues = []
    model.eval()

    with torch.no_grad():
        for sample_batched in test_loader:
            data, target = sample_batched['seq'].to(device), sample_batched['label'].to(device)
            output = model(data)
            pred = [test_loader.dataset.class_names[i] for i in list(output.max(1)[1].cpu().detach().numpy())]
            preds += pred
            true = [test_loader.dataset.class_names[i] for i in list(target.cpu().detach().numpy())]
            trues += true

    labels = test_loader.dataset.class_names

    cm = confusion_matrix(trues, preds, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    cr = classification_report(trues, preds, target_names=labels)
    print(cr)
    correct = 0

    for pred, true in zip(preds, trues):
        if pred == true:
            correct += 1
            
    df = pd.DataFrame({'pred': preds, 'true': trues})

    return correct/len(trues), df

def train_evaluate(train_loader, test_loader, log_interval, num_epoches, seq_length, transform=None, num_channels=72, num_classes=3):

    model = conv1d(num_channels=num_channels, num_classes=num_classes)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    iteration = 0

    for epoch in range(1, 1+num_epoches):
        iteration = train(log_interval, model, device, train_loader, optimizer, epoch, iteration)
        if epoch%10==0:
            test_loss, test_acc = val(model, device, test_loader)
    acc, df = evaluate(model, device, test_loader)

    print(f'Final Acccuracy: {acc}')
    return model

log_interval = 1000
num_epoches = 50
seq_length = 250

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20, shuffle=False)

model = train_evaluate(train_loader, test_loader, log_interval, num_epoches, seq_length, transform)

Validation Accuracy: 25.52%  Test Loss: 3.325800 

Validation Accuracy: 24.48%  Test Loss: 3.922736 

Validation Accuracy: 25.31%  Test Loss: 4.737513 

Validation Accuracy: 25.52%  Test Loss: 5.853477 

Validation Accuracy: 24.48%  Test Loss: 6.195972 

                    precision    recall  f1-score   support

 backside_kickturn       0.35      0.39      0.37       123
frontside_kickturn       0.00      0.00      0.00       115
           pumping       0.33      0.29      0.31       240

          accuracy                           0.24       478
         macro avg       0.23      0.23      0.23       478
      weighted avg       0.26      0.24      0.25       478

Final Acccuracy: 0.24476987447698745
